In [1]:
import streamlit as st
st.set_page_config(layout="wide")
import os
from streamlit_extras.add_vertical_space import add_vertical_space
from dataclasses import dataclass
from tools.new_excel_agent import excel_llm_agent
# Set the title for the Streamlit app
# Create a file uploader in the sidebar
from langchain.callbacks.base import BaseCallbackHandler
import pandas as pd
import openai
from tools.viz_plotly import get_primer,format_question,run_request
import warnings
from st_aggrid import AgGrid
from st_aggrid.grid_options_builder import GridOptionsBuilder
from streamlit_pandas_profiling import st_profile_report
from ydata_profiling import ProfileReport
import plotly.graph_objects as go
import plotly.express as px
import os

from langchain_openai import OpenAI
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()


from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from pydantic import BaseModel, Field
from typing import List
import pandas as pd
from pydantic.config import ConfigDict




c:\Users\skaushik047\Desktop\WORK\DRL\V2\Excel_LLM\drl_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import glob
import os

# Define the path to the directory containing the CSV files
folder_path = '.\data\client\LLM'
df1=pd.read_csv('.\data\client\LLM Data_2.csv')
df2=pd.read_csv('.\data\client\LLM Data.csv')
df3=pd.read_csv('.\data\client\product master_old.csv')



# Use glob to get all CSV file paths in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through the list of CSV files and read each one into a DataFrame
for file in csv_files:
  df = pd.read_csv(file)
  dataframes.append(df)

dataframes.append(df1)
dataframes.append(df2)
dataframes.append(df3)

file_list=[]
file_list.append("LLM Data_2")

file_list.append("LLM Data")

file_list.append("product master")

selected_files=file_list.copy()

selected_dataframes = [dataframes[file_list.index(file)] for file in selected_files]


<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:9: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:9: SyntaxWarning: invalid escape sequence '\d'
C:\Users\skaushik047\AppData\Local\Temp\2\ipykernel_4484\3430145268.py:6: SyntaxWarning: invalid escape sequence '\d'
  folder_path = '.\data\client\LLM'
C:\Users\skaushik047\AppData\Local\Temp\2\ipykernel_4484\3430145268.py:7: SyntaxWarning: invalid escape sequence '\d'
  df1=pd.read_csv('.\data\client\LLM Data_2.csv')
C:\Users\skaushik047\AppData\Local\Temp\2\ipykernel_4484\3430145268.py:8: SyntaxWarning: invalid escape sequence '\d'
  df2=pd.read_csv('.\data\client\LLM Data.csv')
C:\Users\skaushik047\AppData\Local\Temp\2\ipykernel_4484\3430145268.py:9: SyntaxWarning: invalid escape sequence

In [13]:
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY") 
aoai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

llm = AzureChatOpenAI(
    model="gpt-4o",
    deployment_name="gpt-4o",
    api_key=aoai_api_key,
    azure_endpoint=aoai_endpoint,
    api_version=aoai_api_version,
)


# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=aoai_api_key,
    azure_endpoint=aoai_endpoint,
    api_version=aoai_api_version,
)

c:\Users\skaushik047\Desktop\WORK\DRL\V2\Excel_LLM\drl_venv\Lib\site-packages\langchain_openai\embeddings\base.py:212: UserWarning: WARNING! deployment_name is not default parameter.
                    deployment_name was transferred to model_kwargs.
                    Please confirm that deployment_name is what you intended.
  warnings.warn(


In [14]:
llm.invoke("hi")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-2ddb9cd7-f0cb-4b96-bb03-cd6b8b31a236-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17})

## Creating Tools

In [4]:
from typing import Annotated

# from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.tools import PythonREPLTool

# tavily_tool = TavilySearchResults(max_results=5)

# This executes code locally, which can be unsafe
python_repl_tool = PythonREPLTool()

In [8]:
from typing import Annotated

# from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.tools import PythonREPLTool

# tavily_tool = TavilySearchResults(max_results=5)

# This executes code locally, which can be unsafe
python_repl_tool = PythonREPLTool()

In [9]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI


def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [10]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [16]:
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

members = ["Researcher", "Coder"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

# llm = ChatOpenAI(model="gpt-4-1106-preview")

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [24]:
import functools
from langgraph.graph import END, StateGraph, START
import operator
from typing import Sequence, TypedDict

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [26]:
# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION
code_agent = create_agent(
    llm,
    [python_repl_tool],
    "You may generate safe python code to analyze data and generate charts using matplotlib.",
)
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

In [27]:
# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str

pandas_agent = create_pandas_dataframe_agent(
    llm,
    df1,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    # agent_type="openai-tools",
    allow_dangerous_code=True
)
pandas_node = functools.partial(agent_node, agent=pandas_agent, name="Analyst")


workflow = StateGraph(AgentState)
workflow.add_node("Analyst", pandas_node)

for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.set_entry_point("supervisor")

graph = workflow.compile()

for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="summarize the columns in the dataframe provided to you")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

ValueError: Found edge starting at unknown node 'Coder'